# 1. Importación de paquetes

In [136]:
#Import packages
from modules import preprocess
from modules import training

# 2. Preprocesamiento del *dataset*: [Room Occupancy Estimation Data Set](https://www.kaggle.com/ananthr1/room-occupancy-estimation-data-set)

## Paso 1: Lectura del *dataset* y obtención de datos

In [137]:
df = preprocess.load_csv_data('Occupancy_Estimation.csv')
df.shape

(10129, 19)

In [138]:
df.head()

,Date,Time,S1_Temp,S2_Temp,S3_Temp,S4_Temp,S1_Light,S2_Light,S3_Light,S4_Light,S1_Sound,S2_Sound,S3_Sound,S4_Sound,S5_CO2,S5_CO2_Slope,S6_PIR,S7_PIR,Room_Occupancy_Count
0,2017/12/22,10:49:41,24.94,24.75,24.56,25.38,121,34,53,40,0.08,0.19,0.06,0.06,390,0.769231,0,0,1
1,2017/12/22,10:50:12,24.94,24.75,24.56,25.44,121,33,53,40,0.93,0.05,0.06,0.06,390,0.646154,0,0,1
2,2017/12/22,10:50:42,25.00,24.75,24.50,25.44,121,34,53,40,0.43,0.11,0.08,0.06,390,0.519231,0,0,1
3,2017/12/22,10:51:13,25.00,24.75,24.56,25.44,121,34,53,40,0.41,0.10,0.10,0.09,390,0.388462,0,0,1
4,2017/12/22,10:51:44,25.00,24.75,24.56,25.44,121,34,54,40,0.18,0.06,0.06,0.06,390,0.253846,0,0,1


## Paso 2: Recodificación de la columna *RoomOccupancyCount*

En este caso, la columna que contiene la variable de salida (*RoomOccupancyCount*) cuenta el número de personas en la sala. Para simplificar, nuestro objetivo va a ser detectar si la sala está vacía o, en cambio, hay alguien (no el número exacto de personas).

Por lo tanto, se **recodificará cualquier valor mayor que 0 en esa columna como 1**, de tal forma que la variable de salida sea binaria:
- 0 -> ausencia.
- 1 -> presencia.

In [139]:
# Show "Room_Occupancy_Count" before processing
df["Room_Occupancy_Count"]

0        1
1        1
2        1
3        1
4        1
        ..
10124    0
10125    0
10126    0
10127    0
10128    0
Name: Room_Occupancy_Count, Length: 10129, dtype: int64

In [140]:
#Process "Room_Occupancy_Count"
preprocess.recode_dataset_output(df)

# Show "Room_Occupancy_Count" after processing
df["Room_Occupancy_Count"]

0        1
1        1
2        1
3        1
4        1
        ..
10124    0
10125    0
10126    0
10127    0
10128    0
Name: Room_Occupancy_Count, Length: 10129, dtype: int32

## Paso 3: Eliminación de las columnas temporales *Date* y *Time*

Respecto a los datos ofrecidos por las columas *Date* y *Time*, vamos a filtrarlos y dejarlos fuera del proceso. Ya que, aunque los incluyeramos (por ejemplo, juntando ambas partes en una sola columna y convirtiéndolo a formato 'epoch', como long int), los valores de muestras sucesivas de esa columna estarían totalmente correlados entre sí y, como consecuencia, fastidiaríamos a la mayoría de algoritmos que vamos a emplear posteriormente.

Se debe tener en cuenta que no nos estamos olvidando de la información temporal para considerar como si cada valor muestreado (fila de la tabla) fuese independiente de las demás filas. Sabemos que eso no es así, pero estamos considerando que esa información temporal no nos ofrece valor añadido para predecir si la habitación está ocupada o vacía. En caso contrario, se tendrían que usar modelos bastante más complicados para considerar esa relación temporal que indica que, en realidad, las muestras de dos filas adyacentes son consecutivas en el tiempo.

In [141]:
# Filter Date and Time columns
preprocess.remove_time_columns(df)

# Show the result
df.head()

,S1_Temp,S2_Temp,S3_Temp,S4_Temp,S1_Light,S2_Light,S3_Light,S4_Light,S1_Sound,S2_Sound,S3_Sound,S4_Sound,S5_CO2,S5_CO2_Slope,S6_PIR,S7_PIR,Room_Occupancy_Count
0,24.94,24.75,24.56,25.38,121,34,53,40,0.08,0.19,0.06,0.06,390,0.769231,0,0,1
1,24.94,24.75,24.56,25.44,121,33,53,40,0.93,0.05,0.06,0.06,390,0.646154,0,0,1
2,25.00,24.75,24.50,25.44,121,34,53,40,0.43,0.11,0.08,0.06,390,0.519231,0,0,1
3,25.00,24.75,24.56,25.44,121,34,53,40,0.41,0.10,0.10,0.09,390,0.388462,0,0,1
4,25.00,24.75,24.56,25.44,121,34,54,40,0.18,0.06,0.06,0.06,390,0.253846,0,0,1


## Paso 4: División del *dataset* en *train* y *test*

In [142]:
# Split dataset into train and test
train_df, test_df = preprocess.split_dataset(df)
train_df.shape, test_df.shape

((7596, 17), (2533, 17))

In [143]:
train_df.head()

,S1_Temp,S2_Temp,S3_Temp,S4_Temp,S1_Light,S2_Light,S3_Light,S4_Light,S1_Sound,S2_Sound,S3_Sound,S4_Sound,S5_CO2,S5_CO2_Slope,S6_PIR,S7_PIR,Room_Occupancy_Count
4647,25.25,25.31,24.75,25.63,0,0,0,0,0.06,0.05,0.06,0.06,360,0.000000,0,0,0
804,26.25,26.31,26.00,26.31,149,234,178,10,0.50,0.35,2.04,0.47,1015,2.538462,1,1,1
4287,25.38,25.44,24.94,25.75,0,0,0,0,0.07,0.05,0.06,0.06,370,0.000000,0,0,0
5616,25.19,25.19,24.94,25.88,15,17,67,47,0.08,0.05,0.06,0.08,355,0.000000,0,0,0
2469,25.06,25.00,24.50,25.38,6,6,31,21,0.07,0.05,0.06,0.05,360,0.000000,0,0,0


In [144]:
test_df.head()

,S1_Temp,S2_Temp,S3_Temp,S4_Temp,S1_Light,S2_Light,S3_Light,S4_Light,S1_Sound,S2_Sound,S3_Sound,S4_Sound,S5_CO2,S5_CO2_Slope,S6_PIR,S7_PIR,Room_Occupancy_Count
10055,25.13,25.06,24.63,25.13,6,6,30,20,0.08,0.04,0.05,0.09,345,0.000000,0,0,0
5531,25.19,25.19,24.88,25.81,12,14,55,39,0.09,0.10,0.08,0.10,355,0.000000,0,0,0
9401,25.19,25.19,24.75,25.25,0,0,0,0,0.07,0.05,0.07,0.08,345,0.000000,0,0,0
9468,25.19,25.19,24.69,25.31,0,0,0,0,0.07,0.05,0.06,0.08,345,0.000000,0,0,0
3522,26.19,26.13,25.81,26.38,150,237,186,11,0.55,0.10,0.08,0.07,795,1.169231,1,0,1


## Paso 5: División del *dataset* de entrenamiento en *features* y *output*

In [145]:
# Get features columns
X = preprocess.get_features(train_df)

# Get output column
y = preprocess.get_output(train_df)

In [146]:
X.head()

,S1_Temp,S2_Temp,S3_Temp,S4_Temp,S1_Light,S2_Light,S3_Light,S4_Light,S1_Sound,S2_Sound,S3_Sound,S4_Sound,S5_CO2,S5_CO2_Slope,S6_PIR,S7_PIR
4647,25.25,25.31,24.75,25.63,0,0,0,0,0.06,0.05,0.06,0.06,360,0.000000,0,0
804,26.25,26.31,26.00,26.31,149,234,178,10,0.50,0.35,2.04,0.47,1015,2.538462,1,1
4287,25.38,25.44,24.94,25.75,0,0,0,0,0.07,0.05,0.06,0.06,370,0.000000,0,0
5616,25.19,25.19,24.94,25.88,15,17,67,47,0.08,0.05,0.06,0.08,355,0.000000,0,0
2469,25.06,25.00,24.50,25.38,6,6,31,21,0.07,0.05,0.06,0.05,360,0.000000,0,0


In [147]:
y.head()

4647    0
804     1
4287    0
5616    0
2469    0
Name: Room_Occupancy_Count, dtype: int32

# 2. Entrenamiento y seguimiento de la huella de carbono con **CodeCarbon**

In [148]:
# Logistic Regression
training.train_LR_codecarbon(X,y)

[codecarbon INFO @ 11:31:57] [setup] RAM Tracking...
[codecarbon INFO @ 11:31:57] [setup] GPU Tracking...
[codecarbon INFO @ 11:31:57] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 11:31:57] [setup] CPU Tracking...
[codecarbon WARNING @ 11:31:57] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 11:31:59] CPU Model on constant consumption mode: Intel(R) Core(TM) i5-9400F CPU @ 2.90GHz
[codecarbon INFO @ 11:31:59] >>> Tracker's metadata:
[codecarbon INFO @ 11:31:59]   Platform system: Windows-10-10.0.19044-SP0
[codecarbon INFO @ 11:31:59]   Python version: 3.10.4
[codecarbon INFO @ 11:31:59]   Available RAM : 7.935 GB
[codecarbon INFO @ 11:31:59]   CPU count: 6
[codecarbon INFO @ 11:31:59]   CPU model: Intel(R) Core(TM) i5-9400F CPU @ 2.90GHz
[codecarbon INFO @ 11:31:59]   GPU count: 1
[codecarbon INFO @ 11:31:59]   GPU model: 1 x NVIDIA GeForce GT 710
[codecarbon INFO @ 11:31:59] Energy consumed for RAM : 0.000000 kWh. RAM Power : 2.97562837600708 W


Emissions: 1.190639492866931e-07 kg


In [149]:
# Random Forest
training.train_RF_codecarbon(X,y)

[codecarbon INFO @ 11:31:59] [setup] RAM Tracking...
[codecarbon INFO @ 11:31:59] [setup] GPU Tracking...
[codecarbon INFO @ 11:31:59] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 11:31:59] [setup] CPU Tracking...
[codecarbon WARNING @ 11:31:59] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 11:32:01] CPU Model on constant consumption mode: Intel(R) Core(TM) i5-9400F CPU @ 2.90GHz
[codecarbon INFO @ 11:32:01] >>> Tracker's metadata:
[codecarbon INFO @ 11:32:01]   Platform system: Windows-10-10.0.19044-SP0
[codecarbon INFO @ 11:32:01]   Python version: 3.10.4
[codecarbon INFO @ 11:32:01]   Available RAM : 7.935 GB
[codecarbon INFO @ 11:32:01]   CPU count: 6
[codecarbon INFO @ 11:32:01]   CPU model: Intel(R) Core(TM) i5-9400F CPU @ 2.90GHz
[codecarbon INFO @ 11:32:01]   GPU count: 1
[codecarbon INFO @ 11:32:01]   GPU model: 1 x NVIDIA GeForce GT 710
[codecarbon INFO @ 11:32:02] Energy consumed for RAM : 0.000000 kWh. RAM Power : 2.97562837600708 W


Emissions: 4.2526682692106333e-07 kg


In [150]:
# Linear SVC (Support Vector Machines)
training.train_SVC_codecarbon(X,y)

[codecarbon INFO @ 11:32:02] [setup] RAM Tracking...
[codecarbon INFO @ 11:32:02] [setup] GPU Tracking...
[codecarbon INFO @ 11:32:02] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 11:32:02] [setup] CPU Tracking...
[codecarbon WARNING @ 11:32:02] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 11:32:04] CPU Model on constant consumption mode: Intel(R) Core(TM) i5-9400F CPU @ 2.90GHz
[codecarbon INFO @ 11:32:04] >>> Tracker's metadata:
[codecarbon INFO @ 11:32:04]   Platform system: Windows-10-10.0.19044-SP0
[codecarbon INFO @ 11:32:04]   Python version: 3.10.4
[codecarbon INFO @ 11:32:04]   Available RAM : 7.935 GB
[codecarbon INFO @ 11:32:04]   CPU count: 6
[codecarbon INFO @ 11:32:04]   CPU model: Intel(R) Core(TM) i5-9400F CPU @ 2.90GHz
[codecarbon INFO @ 11:32:04]   GPU count: 1
[codecarbon INFO @ 11:32:04]   GPU model: 1 x NVIDIA GeForce GT 710
C:\Users\INTEL I5\miniconda3\envs\TFG_Project\lib\site-packages\sklearn\svm\_base.py:1225: Convergen

Emissions: 5.616847085585006e-08 kg


In [151]:
!pip install experiment-impact-tracker

In [152]:
import tempfile
from experiment_impact_tracker.compute_tracker import ImpactTracker
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from codecarbon import EmissionsTracker
from cumulator import base

tmp_dir = tempfile.mkdtemp()
# Init tracker with log path
tracker = ImpactTracker(tmp_dir)
# Start tracker in a separate process
tracker.launch_impact_monitor()
lg_pipeline = Pipeline([("scaler", StandardScaler()), ("logistic_regression", LogisticRegression())])
lg_pipeline.fit(X, y)

loading region bounding boxes for computing carbon emissions region, this may take a moment...
 454/454... rate=522.09 Hz, eta=0:00:00, total=0:00:000
Done!


UnicodeDecodeError: 'charmap' codec can't decode byte 0x8d in position 14279: character maps to <undefined>